In [ ]:
import pandas as pd
import re
from transformers import pipeline

# Load dataset
data = pd.read_csv('Restaurant_reviews.csv')
data = data[['Review', 'Rating']].dropna().reset_index(drop=True)

# Initialize columns for each aspect
data['Food Quality Sentiment'] = ''
data['Service Sentiment'] = ''
data['Ambiance Sentiment'] = ''

# Define keywords for each aspect
food_keywords = ['food', 'taste', 'flavor', 'meal', 'dish', 'cuisine']
service_keywords = ['service', 'staff', 'waiter', 'waitress', 'attentive', 'slow', 'friendly']
ambiance_keywords = ['ambiance', 'atmosphere', 'environment', 'decor', 'vibe', 'music']

# Function to extract aspect-specific text based on keywords
def extract_aspect_text(review, keywords):
    sentences = re.split(r'[.!?]', review)  # Split review into sentences
    aspect_text = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in keywords)]
    return ' '.join(aspect_text)

# Apply extraction for each aspect
data['Food Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, food_keywords))
data['Service Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, service_keywords))
data['Ambiance Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, ambiance_keywords))

# Initialize a sentiment analysis pipeline using a model with "neutral" classification
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", truncation=True)

# Adjusted function for analyzing sentiment with thresholds
def analyze_sentiment(aspect_text):
    # Limit input to 512 characters to avoid model limit issues
    aspect_text = aspect_text[:512] if len(aspect_text) > 512 else aspect_text
    
    if aspect_text.strip():  # Ensure there's text to analyze
        result = sentiment_analyzer(aspect_text)[0]
        sentiment = result['label'].lower()
        score = result['score']

        # Map sentiment based on score thresholds
        if sentiment == "positive" and score > 0.75:
            sentiment = "very positive"
        elif sentiment == "positive":
            sentiment = "positive"
        elif sentiment == "negative" and score > 0.75:
            sentiment = "very negative"
        elif sentiment == "negative":
            sentiment = "negative"
        else:
            sentiment = "neutral"  # Assign "neutral" for uncertain scores
    else:
        sentiment = 'neutral'  # If no aspect text was extracted
    
    return sentiment

# Apply the sentiment analysis to each aspect
data['Food Quality Sentiment'] = data['Food Text'].apply(analyze_sentiment)
data['Service Sentiment'] = data['Service Text'].apply(analyze_sentiment)
data['Ambiance Sentiment'] = data['Ambiance Text'].apply(analyze_sentiment)

# Display a sample to verify the results
print(data[['Review', 'Food Quality Sentiment', 'Service Sentiment', 'Ambiance Sentiment']].head())

# Save the dataset with aspect sentiments to a CSV file
data[['Review', 'Food Quality Sentiment', 'Service Sentiment', 'Ambiance Sentiment']].to_csv('Restaurant_Reviews_with_Aspect_Sentiment2.csv', index=False)

print("Processed data saved as 'Restaurant_Reviews_with_Aspect_Sentiment.csv'")


C:\Users\Iflal\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                              Review Food Quality Sentiment  \
0  The ambience was good, food was quite good . h...                neutral   
1  Ambience is too good for a pleasant evening. S...                neutral   
2  A must try.. great food great ambience. Thnx f...                neutral   
3  Soumen das and Arun was a great guy. Only beca...                neutral   
4  Food is good.we ordered Kodi drumsticks and ba...                neutral   

  Service Sentiment Ambiance Sentiment  
0           neutral            neutral  
1           neutral            neutral  
2           neutral            neutral  
3           neutral            neutral  
4           neutral            neutral  
Processed data saved as 'Restaurant_Reviews_with_Aspect_Sentiment.csv'
